In [27]:
import math
class ConvexHull:
    def __init__(self, points):
        self.hull = self.convex_hull_2d(points)
    
    def convex_hull_2d(self, points):
        if len(points) < 5:
            return self.find_convex_hull(points)
        else:
            midpoint = int(len(points)/2)
            left_hull = self.convex_hull_2d(points[:midpoint])
            right_hull = self.convex_hull_2d(points[midpoint:])

        return self.merge_hull(left_hull, right_hull)
        
    # Finds upper tangent of two polygons 'lower_hull' and 'upper_hull' represented as two vectors.
    def merge_hull(self, lower_hull, upper_hull):
        p = len(lower_hull)
        q = len(upper_hull)

        leftmost1 = lower_hull.index(max(lower_hull, key=lambda x: x[0]))
        leftmost2 = upper_hull.index(min(upper_hull, key=lambda x: x[0]))

        check_tangent = lambda p1, upper_hull, c: (upper_hull[1]-p1[1]) * (c[0]-upper_hull[0]) - (c[1]-upper_hull[1]) * (upper_hull[0]-p1[0])

        # find upper tangent
        i = next((i for i in range(p) if check_tangent(upper_hull[leftmost2], lower_hull[leftmost1], lower_hull[(leftmost1+1) % p]) >= 0), leftmost1)
        j = next((j for j in range(q) if check_tangent(lower_hull[i], upper_hull[leftmost2], upper_hull[(q+leftmost2-1) % q]) <= 0), leftmost2)

        uppera, upperb = i, j

        # find lower tangent
        j = next((j for j in range(p) if check_tangent(lower_hull[leftmost1], upper_hull[leftmost2], upper_hull[(leftmost2+1) % q]) >= 0), leftmost2)
        i = next((i for i in range(q) if check_tangent(upper_hull[j], lower_hull[leftmost1], lower_hull[(p+leftmost1-1) % p]) <= 0), leftmost1)

        lowera, lowerb = i, j

        final = []
        if lowera < uppera:
            final = lower_hull[uppera:lowera+1] + upper_hull[lowerb:upperb+1]
        else:
            final = upper_hull[lowerb:upperb+1] + lower_hull[uppera:lowera+1]
        return final
        
    def find_convex_hull_brute_force(self, points):
        n = len(points)
        if n <= 1:
            return points
        elif n == 2:
            return points
        elif n == 3:
            a, b, c = points
            if (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0]) == 0:
                return [a, c]
            else:
                return points
        elif n == 4:
            return find_convex_hull_graham(points)
        else:
            hull = []
            for i in range(n):
                for j in range(i + 1, n):
                    line = [points[i], points[j]]
                    upper, lower = [], []
                    for k in range(n):
                        if k == i or k == j:
                            continue
                        elif is_above(line, points[k]):
                            upper.append(points[k])
                        else:
                            lower.append(points[k])
                    if len(upper) == 0 or len(lower) == 0:
                        continue
                    upper_hull = find_convex_hull_brute_force(upper)
                    lower_hull = find_convex_hull_brute_force(lower)
                    hull += upper_hull + lower_hull
            return find_convex_hull_graham(hull)
        

    def is_above(self,line, point):
        a, b = line
        return (point[1] - a[1]) * (b[0] - a[0]) > (point[0] - a[0]) * (b[1] - a[1])

    def convex_hull_brute(self, points):
    # Base case: return all points if there are only three or fewer
        if len(points) <= 3:
            return points

        # Find all pairs of points and calculate their slopes
        pairs = [(points[i], points[j]) for i in range(len(points)) for j in range(i+1, len(points))]
        slopes = [(p2[1]-p1[1])/(p2[0]-p1[0]) if p2[0] != p1[0] else float('inf') for (p1, p2) in pairs]

        # Find the pair with the smallest slope, which must be on the convex hull
        min_slope_idx = slopes.index(min(slopes))
        p1, p2 = pairs[min_slope_idx]

        # Partition the points into those on the left and right of the line between p1 and p2
        left = [p for p in points if (p[1]-p1[1])*(p2[0]-p1[0]) < (p[0]-p1[0])*(p2[1]-p1[1])]
        right = [p for p in points if (p[1]-p1[1])*(p2[0]-p1[0]) >= (p[0]-p1[0])*(p2[1]-p1[1])]

        # Recursively find the convex hulls of the left and right partitions
        left_hull = self.convex_hull_brute(left)
        right_hull = self.convex_hull_brute(right)

        # Combine the convex hulls of the left and right partitions to form the final convex hull
        return left_hull + right_hull
    
    def find_convex_hull(self, hull):
        def find_corner(p1, q1):
            p = [p1[0] - reference_point[0], p1[1] - reference_point[1]]
            q = [q1[0] - reference_point[0], q1[1] - reference_point[1]]
            cross_product = p[0] * q[1] - p[1] * q[0]
            if cross_product > 0:
                return -1
            elif cross_product < 0:
                return 1
            else:
                return 0
    
        # Find the lower hull of the points

        s = set()
        n = len(hull)
        for i in range(n):
            for j in range(i+1, n):
                points = [hull[k] for k in range(n) if k != i and k != j]
                x1, y1 = hull[i]
                x2, y2 = hull[j]
                a, b, c = y1-y2, x2-x1, x1*y2-y1*x2
                pos = sum(1 for (x, y) in points if a*x+b*y+c >= 0)
                neg = sum(1 for (x, y) in points if a*x+b*y+c <= 0)
                if pos == n-2 or neg == n-2:
                    s.update({hull[i], hull[j]})

        final = [list(x) for x in s]

        # counterclockwise sort
        reference_point = [0, 0]
        n = len(final)
        reference_point = [sum(p[0] for p in final), sum(p[1] for p in final)]  # calculate reference point
        final = [[n*final[i][0], n*final[i][1]] for i in range(n)]

        from functools import cmp_to_key
        final = sorted(final, key=cmp_to_key(find_corner))
        for i in range(n):
            final[i] = [final[i][0]/n, final[i][1]/n]
        return final
    
    import math

    def find_convex_hull_brute_force(self, hull_points):

        n = len(hull_points)
        if n < 3:
            return hull_points
        
        def is_point_on_right_of_line(line, point):
            x1, y1 = line[0]
            x2, y2 = line[1]
            x, y = point
            return (y - y1) * (x2 - x1) > (x - x1) * (y2 - y1)
    
        hull = []
        for i in range(n):
            for j in range(i+1, n):
                line = (hull_points[i], hull_points[j])
                points_on_right = []
                for k in range(n):
                    if k != i and k != j and is_point_on_right_of_line(line, hull_points[k]):
                        points_on_right.append(hull_points[k])
                if not points_on_right:
                    if hull_points[i] not in hull:
                        hull.append(hull_points[i])
                    if hull_points[j] not in hull:
                        hull.append(hull_points[j])
                else:
                    # sort points on right of line by polar angle w.r.t. reference point
                    reference_point = hull_points[i]
                    points_on_right.sort(key=lambda p: math.atan2(p[1]-reference_point[1], p[0]-reference_point[0]))
                    if hull_points[i] not in hull:
                        hull.append(hull_points[i])
                    if points_on_right[-1] not in hull:
                        hull.append(points_on_right[-1])
        return hull


In [28]:
if __name__ == '__main__':
    hull = ConvexHull(sorted([(1,2),(9,10),(19,10),(8,19),(4,6),(7,5)]))

    print('Convex Hull:')
    for x in hull.hull:
        print(int(x[0]), int(x[1]))

Convex Hull:
8 19
1 2
4 6
7 5
